In [1]:
# import opensim as osim
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from data_utils import *
from eval_utils import *

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

from lstm import LSTMModel
from cnnlstm import CNNLSTMModel
from lstmattn import LSTMAttentionModel
from transformer import TransformerModel

In [2]:
# define device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


In [3]:
data_dir = '../data/'

test_data = np.load(data_dir + 'test_data.npz')

grf_labels = ['GRF_x', 'GRF_y', 'GRF_z']
muscle_labels = ['tibpost', 'tibant', 'edl', 'ehl', 'fdl', 'fhl', 'perbrev', 'perlong', 'achilles']

grf_dict = {0: 'GRF_x', 1: 'GRF_y', 2: 'GRF_z'}
muscle_dict = {0: 'tibpost', 1: 'tibant', 2: 'edl', 3: 'ehl', 4: 'fdl', 5: 'fhl', 6: 'perbrev', 7: 'perlong', 8: 'achilles'}

In [4]:
X_test = test_data['X_test']
y_test = test_data['y_test']

print(X_test.shape, y_test.shape)

(1340, 100, 3) (1340, 100, 9)


In [5]:
# convert test data to torch tensors
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).to(device)

In [6]:
lstm_model = LSTMModel(input_size=3, 
                       hidden_size=256, 
                       num_layers=2, 
                       output_size=9, 
                       dropout=0.08941182342943683)

lstm_model.load_state_dict(torch.load('../models/lstm.pth', weights_only=True))

lstm_model.to(device)

# criterion = nn.MSELoss()
# optimizer = optim.Adam(lstm_model.parameters(), 
#                        lr=0.0004070688993179255, 
#                        weight_decay=4.278679617413207e-05)

LSTMModel(
  (lstm): LSTM(3, 256, num_layers=2, batch_first=True, dropout=0.08941182342943683)
  (fc): Linear(in_features=256, out_features=9, bias=True)
)

In [7]:
lstm_loss, lstm_pred_tensor = eval_model(lstm_model, X_test_tensor, y_test_tensor)
print(f"LSTM Loss: {lstm_loss}")

LSTM Loss: 1532.4300537109375


In [8]:
# y_test = y_test_tensor.cpu().numpy()
lstm_pred = lstm_pred_tensor.cpu().numpy()

lstm_rrmse = calc_rrmse_muscle(y_test, lstm_pred)

tibpost: 0.0755
tibant: 0.0343
edl: 0.0521
ehl: 0.0307
fdl: 0.0676
fhl: 0.0790
perbrev: 0.0532
perlong: 0.0465
achilles: 0.0222


In [26]:
lstm_r2 = calc_r2_muscle(y_test, lstm_pred)

tibpost: 0.8238
tibant: 0.9358
edl: 0.8328
ehl: 0.9285
fdl: 0.8103
fhl: 0.7824
perbrev: 0.7212
perlong: 0.6422
achilles: 0.9910


In [9]:
cnnlstm_model = CNNLSTMModel(input_size=3, 
                             hidden_size=256,
                             num_layers=3, 
                             output_size=9, 
                             dropout=0.17575462419723403)

cnnlstm_model.load_state_dict(torch.load('../models/cnn-lstm.pth', weights_only=True))

cnnlstm_model.to(device)

CNNLSTMModel(
  (cnn): Sequential(
    (0): Conv1d(3, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): MaxPool1d(kernel_size=2, stride=1, padding=1, dilation=1, ceil_mode=False)
    (3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ReLU()
    (5): MaxPool1d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (lstm): LSTM(128, 256, num_layers=3, batch_first=True, dropout=0.17575462419723403)
  (fc): Linear(in_features=256, out_features=9, bias=True)
)

In [10]:
cnnlstm_loss, cnnlstm_pred_tensor = eval_model(cnnlstm_model, X_test_tensor, y_test_tensor)
print(f"CNN-LSTM Loss: {cnnlstm_loss}")

CNN-LSTM Loss: 1332.114501953125


In [11]:
# y_test = y_test_tensor.cpu().numpy()
cnnlstm_pred = cnnlstm_pred_tensor.cpu().numpy()

cnnlstm_rrmse = calc_rrmse_muscle(y_test, cnnlstm_pred)

tibpost: 0.0716
tibant: 0.0302
edl: 0.0404
ehl: 0.0252
fdl: 0.0641
fhl: 0.0751
perbrev: 0.0467
perlong: 0.0409
achilles: 0.0207


In [27]:
cnnlstm_r2 =calc_r2_muscle(y_test, cnnlstm_pred)

tibpost: 0.8417
tibant: 0.9505
edl: 0.8995
ehl: 0.9518
fdl: 0.8298
fhl: 0.8032
perbrev: 0.7851
perlong: 0.7237
achilles: 0.9922


In [12]:
lstmattn_model = LSTMAttentionModel(input_size=3, 
                                    hidden_size=256, 
                                    num_layers=3, 
                                    num_heads=8, 
                                    output_size=9, 
                                    lstm_dropout=0.08509082023364795, 
                                    attn_dropout=0.3104457129518861)

lstmattn_model.load_state_dict(torch.load('../models/lstm-attn.pth', weights_only=True))

lstmattn_model.to(device)

LSTMAttentionModel(
  (lstm): LSTM(3, 256, num_layers=3, batch_first=True, dropout=0.08509082023364795)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
  )
  (fc): Linear(in_features=256, out_features=9, bias=True)
)

In [13]:
lstmattn_loss, lstmattn_pred_tensor = eval_model(lstmattn_model, X_test_tensor, y_test_tensor)
print(f"LSTM-Attention Loss: {lstmattn_loss}")

LSTM-Attention Loss: 979.2155151367188


In [14]:
lstmattn_pred = lstmattn_pred_tensor.cpu().numpy()

lstmattn_rrmse = calc_rrmse_muscle(y_test, lstmattn_pred)

tibpost: 0.0637
tibant: 0.0268
edl: 0.0385
ehl: 0.0282
fdl: 0.0657
fhl: 0.0683
perbrev: 0.0473
perlong: 0.0408
achilles: 0.0169


In [25]:
lstmattn_r2= calc_r2_muscle(y_test, lstmattn_pred)

tibpost: 0.8747
tibant: 0.9608
edl: 0.9087
ehl: 0.9396
fdl: 0.8208
fhl: 0.8372
perbrev: 0.7796
perlong: 0.7248
achilles: 0.9948


In [15]:
transformer_model = TransformerModel(input_dim=3,
                                     output_dim=9, 
                                     d_model=32, 
                                     nhead=8, 
                                     num_encoder_layers=6, 
                                     dim_feedforward=256, 
                                     dropout=0.012122943820592716)

transformer_model.load_state_dict(torch.load('../models/transformer.pth', weights_only=True))

transformer_model.to(device)

TransformerModel(
  (input_embedding): Linear(in_features=3, out_features=32, bias=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
        )
        (linear1): Linear(in_features=32, out_features=256, bias=True)
        (dropout): Dropout(p=0.012122943820592716, inplace=False)
        (linear2): Linear(in_features=256, out_features=32, bias=True)
        (norm1): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.012122943820592716, inplace=False)
        (dropout2): Dropout(p=0.012122943820592716, inplace=False)
      )
    )
  )
  (output_layer): Linear(in_features=32, out_features=9, bias=True)
)

In [16]:
transformer_loss, transformer_pred_tensor = eval_model(transformer_model, X_test_tensor, y_test_tensor)
print(f"Transformer Loss: {transformer_loss}")

Transformer Loss: 890.1572265625


In [17]:
transformer_pred = transformer_pred_tensor.cpu().numpy()

transformer_rrmse = calc_rrmse_muscle(y_test, transformer_pred)

tibpost: 0.0574
tibant: 0.0251
edl: 0.0313
ehl: 0.0244
fdl: 0.0534
fhl: 0.0607
perbrev: 0.0431
perlong: 0.0361
achilles: 0.0170


In [18]:
transformer_r2 = calc_r2_muscle(y_test, transformer_pred)

tibpost: 0.8982
tibant: 0.9659
edl: 0.9397
ehl: 0.9547
fdl: 0.8817
fhl: 0.8717
perbrev: 0.8169
perlong: 0.7846
achilles: 0.9947


In [19]:
lstm_rmse_overall = calc_rmse_overall(y_test, lstm_pred)
print(f"LSTM RMSE Overall: {lstm_rmse_overall}")

cnnlstm_rmse_overall = calc_rmse_overall(y_test, cnnlstm_pred)
print(f"CNN-LSTM RMSE Overall: {cnnlstm_rmse_overall}")

lstmattn_rmse_overall = calc_rmse_overall(y_test, lstmattn_pred)
print(f"LSTM-Attention RMSE Overall: {lstmattn_rmse_overall}")

transformer_rmse_overall = calc_rmse_overall(y_test, transformer_pred)
print(f"Transformer RMSE Overall: {transformer_rmse_overall}")

LSTM RMSE Overall: 39.14626374012921
CNN-LSTM RMSE Overall: 36.498143321045575
LSTM-Attention RMSE Overall: 31.292418308225095
Transformer RMSE Overall: 29.835502999717793


In [20]:
lstm_rrmse_overall = calc_rrmse_overall(y_test, lstm_pred)
print(f"LSTM RRMSE Overall: {lstm_rrmse_overall}")

cnnlstm_rrmse_overall = calc_rrmse_overall(y_test, cnnlstm_pred)
print(f"CNN-LSTM RRMSE Overall: {cnnlstm_rrmse_overall}")

lstmattn_rrmse_overall = calc_rrmse_overall(y_test, lstmattn_pred)
print(f"LSTM-Attention RRMSE Overall: {lstmattn_rrmse_overall}")

transformer_rrmse_overall = calc_rrmse_overall(y_test, transformer_pred)
print(f"Transformer RRMSE Overall: {transformer_rrmse_overall}")

LSTM RRMSE Overall: 0.010057245780650328
CNN-LSTM RRMSE Overall: 0.009376905043963846
LSTM-Attention RRMSE Overall: 0.008039478405550209
Transformer RRMSE Overall: 0.007665175625685436


In [21]:
lstm_rrmse_weighted = calc_rrmse_weighted(y_test, lstm_pred)
print(f"LSTM RRMSE Weighted: {lstm_rrmse_weighted}")

cnnlstm_rrmse_weighted = calc_rrmse_weighted(y_test, cnnlstm_pred)
print(f"CNN-LSTM RRMSE Weighted: {cnnlstm_rrmse_weighted}")

lstmattn_rrmse_weighted = calc_rrmse_weighted(y_test, lstmattn_pred)
print(f"LSTM-Attention RRMSE Weighted: {lstmattn_rrmse_weighted}")

transformer_rrmse_weighted = calc_rrmse_weighted(y_test, transformer_pred)
print(f"Transformer RRMSE Weighted: {transformer_rrmse_weighted}")

LSTM RRMSE Weighted: 0.03625386730483253


CNN-LSTM RRMSE Weighted: 0.03331635888064981
LSTM-Attention RRMSE Weighted: 0.0294192024535717
Transformer RRMSE Weighted: 0.027417547690643182


In [22]:
lstm_rmspe = calc_rmspe_overall(y_test, lstm_pred)
print(f"LSTM RMSPE: {lstm_rmspe}")

cnnlstm_rmspe = calc_rmspe_overall(y_test, cnnlstm_pred)
print(f"CNN-LSTM RMSPE: {cnnlstm_rmspe}")

lstmattn_rmspe = calc_rmspe_overall(y_test, lstmattn_pred)
print(f"LSTM-Attention RMSPE: {lstmattn_rmspe}")

transformer_rmspe = calc_rmspe_overall(y_test, transformer_pred)
print(f"Transformer RMSPE: {transformer_rmspe}")

LSTM RMSPE: 0.4455128557411234
CNN-LSTM RMSPE: 0.4417430053560954
LSTM-Attention RMSPE: 0.43236024565927444
Transformer RMSPE: 0.3589254023390982


In [23]:
lstm_r2 = calc_r2_overall(y_test, lstm_pred)
print(f"LSTM R2: {lstm_r2}")

cnnlstm_r2 = calc_r2_overall(y_test, cnnlstm_pred)
print(f"CNN-LSTM R2: {cnnlstm_r2}")

lstmattn_r2 = calc_r2_overall(y_test, lstmattn_pred)
print(f"LSTM-Attention R2: {lstmattn_r2}")

transformer_r2 = calc_r2_overall(y_test, transformer_pred)
print(f"Transformer R2: {transformer_r2}")

LSTM R2: 0.9934202175245972
CNN-LSTM R2: 0.994280310605302
LSTM-Attention R2: 0.9957955503590777
Transformer R2: 0.9961779388698756


In [24]:
# sample_idx = 0

# true = y_test_tensor[sample_idx].cpu().numpy()
# pred = lstm_pred_tensor[sample_idx].cpu().numpy()

# perc_stance = np.linspace(0, 1, 100)

# fig, axes = plt.subplots(3, 3, figsize=(15, 10))  # Create subplots for 9 muscles
# axes = axes.flatten()

# for i in range(9):
#     axes[i].plot(perc_stance, true[:, i], label="True")
#     axes[i].plot(perc_stance, pred[:, i], label="Predicted", linestyle='dashed')
#     axes[i].set_title(muscle_dict[i])
#     axes[i].set_xlabel("Percent Normalized Stance (%)")
#     axes[i].set_ylabel("Muscle Force (N)")
#     axes[i].legend()

# plt.tight_layout()
# plt.show()